In [50]:
from scamp import test_run, Session, wait
import random
import time
from pythonosc import udp_client
from src.config import ARTIFACTS_RELPATH
from src.scamp_utils import set_default_scamp_settings

set_default_scamp_settings()

## Using Scamp's osc methods

In [101]:
def start_session():
    touch_session = Session()
    touch_osc_part = touch_session.new_osc_part(
        name="touch_osc",
        ip_address="touch_osc",
        port=9000,
    )
    return touch_session, touch_osc_part


def test_run(
    touch_osc_session, touch_osc_part, lowest, highest, step, intervals, repeat_count = 1
):
    touch_osc_session.tempo = 40
    touch_osc_session.start_transcribing()
    sequence = [
        *range(lowest, highest, step),
        *range(highest, lowest, -1 * step),
    ]
    print("Running sequence: ", sequence)
    for repeat in range(repeat_count):
        print("Repeat: ", repeat)
        for pitch in sequence:
            length = 0.1
            velocity = pitch + 30
            for interval in intervals:
                touch_osc_part.play_note(
                    pitch + interval, velocity, length, blocking=False
                )
            wait(length)

    tr = touch_osc_session.stop_transcribing()
    tr.save_to_json(f"{ARTIFACTS_RELPATH}/touch_osc.json")

In [102]:
touch_osc_session, touch_osc_part = start_session()

test_run(touch_osc_session, touch_osc_part, 50, 80, 2, [0, 4, 7], 10)

Running sequence:  [50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 78, 76, 74, 72, 70, 68, 66, 64, 62, 60, 58, 56, 54, 52]
Repeat:  0
Repeat:  1
Repeat:  2
Repeat:  3
Repeat:  4
Repeat:  5


KeyboardInterrupt: 

## Using python osc

Scamps's osc feature seems to have been developed with only ScampUtils of supercollider in mind and does not communicate directly with touchosc's natural way of doing things. 

Until a proper script in lua (or scamp) is developed, python-osc helps with experimentation.


In [10]:
client = udp_client.SimpleUDPClient("touch_osc", 9000)

In [11]:
lowest = 50
highest = 60
sequence = [*range(lowest, highest, 2), *reversed(range(lowest, highest, 2))]
print(sequence)
for _ in range(1):
    for x in sequence:
        address = "/touch_osc"
        # pitch = x * 2 + 50
        pitches = [
            # x * 2 + 50 + 2,
            x
            # x * 2 + 40,
        ]
        # velocity = x * 10
        for pitch in pitches:
            client.send_message(f"{address}/start_note", [pitch, 127])
        time.sleep(0.3)
        for pitch in pitches:
            client.send_message(f"{address}/end_note", [pitch, 127])
        time.sleep(0.5)

[50, 52, 54, 56, 58, 58, 56, 54, 52, 50]
